In [1]:
# from glob import glob
# import datetime as dt
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')

In [2]:
import xarray as xr
import json
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import useful_functions as uf

In [3]:
params = json.load(open(
    os.path.join('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow',
                 'outputs/dbpm_size_params.json')))

In [4]:
dbpm_input = pd.read_parquet(
    os.path.join('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/monthly_weighted',
                 'dbpm_clim-fish-inputs_fao-58_1841-2010.parquet'))

## Parameters from `sizemodel` function

In [5]:
ERSEM_det_input = False
temp_effect = True
use_init = False

In [6]:
# Input parameters to vary:
# time series of intercept of plankton size spectrum (estimated from GCM, 
# biogeophysical model output or satellite data).		
ui0 = 10**np.array(params['int_phy_zoo'])
[numb_size_bins] = params['numb_size_bins']
#Size bin index
size_bin_index = np.arange(0, numb_size_bins)
#Size bins
log10_size_bins = np.array(params['log10_size_bins'])
size_bin_vals = 10**log10_size_bins
[log10_pred_prey_ratio] = params['log10_pred_prey_ratio']
[log_prey_pref] = params['log_prey_pref']
[metabolic_req_pred] = params['metabolic_req_pred']
#Index for minimum detritivore size
[ind_min_detritivore_size] = params['ind_min_detritivore_size']
#Index for minimum predator size
[ind_min_pred_size] = params['ind_min_pred_size']
#Index for minimum size of detritivore fished
[ind_min_fish_det] = params['ind_min_fish_det']
#Index for minimum size of predator fished
[ind_min_fish_pred] = params['ind_min_fish_pred']
#Time
time = pd.date_range(start = min(dbpm_input.time)-DateOffset(months = 1),
                     end = max(dbpm_input.time), freq = 'MS')

# Initialising variables to store DBPM outputs

In [154]:
#data arrays for recording the two size spectra (V - det, U - pred)
predators = uf.init_da(log10_size_bins, time)
detritivores = uf.init_da(log10_size_bins, time)

#vector to hold detritus biomass density (g.m-2)
detritus = xr.DataArray(data = np.zeros(len(time)),
                        dims = 'time',
                        coords = {'time': time})

#data arrays for keeping track of growth (GG_v, GG_u) and reproduction (R_v, R_u) 
#from ingested food:
reprod_det = uf.init_da(log10_size_bins, time)
reprod_pred = uf.init_da(log10_size_bins, time)
growth_det = uf.init_da(log10_size_bins, time)
growth_int_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of predation mortality (PM_v, PM_u)
pred_mort_det = uf.init_da(log10_size_bins, time)
pred_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of catches (Y_v, Y_u)
catch_det = uf.init_da(log10_size_bins, time)
catch_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of total mortality (Z_v, Z_u)
tot_mort_det = uf.init_da(log10_size_bins, time)
tot_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays to hold fishing mortality rates at each size class at time 
#(Fvec_v, Fvec_u)
fishing_mort_det = uf.init_da(log10_size_bins, time)
fishing_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of senescence mortality (SM_v, SM_u) and other 
#(intrinsic) mortality (OM_v, OM_u)
senes_mort_det = np.zeros(len(log10_size_bins))
senes_mort_pred = np.zeros(len(log10_size_bins))
other_mort_det = np.zeros(len(log10_size_bins))
other_mort_pred = np.zeros(len(log10_size_bins))

# Building lookup tables

In [155]:
#lookup tables for terms in the integrals which remain constant over time (gphi, mphi)
constant_growth = uf.gphi_f(uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                            log_prey_pref)
constant_mortality = uf.mphi_f(-uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                               log_prey_pref, metabolic_req_pred)

#lookup table for components of 10^(metabolic_req_pred*log10_size_bins) (expax)
met_req_log10_size_bins = uf.expax_f(log10_size_bins, metabolic_req_pred)

# Numerical integration

In [156]:
# set up with the initial values from param - same for all grid cells
#(phyto+zoo)plankton size spectrum (U) 
predators.isel(size_class = slice(None, 120)).loc[{'time': predators.time.min()}] = \
params['plank_pred_sizes'][:120]

# set initial detritivore spectrum (V)
detritivores.isel(size_class = slice((ind_min_detritivore_size-1), 120)).\
loc[{'time': detritivores.time.min()}] = \
np.array(params['detritivore_sizes'])[ind_min_detritivore_size-1:120]

# set initial detritus biomass density (g.m^-3) (W)
detritus.loc[{'time': detritus.time.min()}] = params['init_detritus'][0]

In [157]:
if use_init:
    # set up with the initial values from previous run
    predators.isel(size_class = slice((ind_min_pred_size-1), None)).\
    loc[{'time': predators.time.min()}] = \
    params['plank_pred_sizes'][(ind_min_pred_size-1):]
    
    # set initial detritivore spectrum from previous run
    detritivores.isel(size_class = slice((ind_min_detritivore_size-1), None)).\
    loc[{'time': detritivores.time.min()}] = \
    np.array(params['detritivore_sizes'])[(ind_min_detritivore_size-1):]

In [158]:
#intrinsic natural mortality (OM.u, OM.v)
other_mort_det = params['natural_mort']*10**(-0.25*log10_size_bins)
other_mort_pred = params['natural_mort']*10**(-0.25*log10_size_bins)

#senescence mortality rate to limit large fish from building up in the system
#same function as in Law et al 2008, with chosen parameters gives similar M2 values
#as in Hall et al. 2006 (SM.u, SM.v)
senes_mort_det = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

senes_mort_pred = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

## Fishing mortality (FVec.u, FVec.v)
From Benoit & Rochet 2004. Here `fish_mort_pred` and `fish_mort_pred` equal fixed catchability term for predators and detritivores. To be estimated along with `ind_min_det` and `ind_min_fish_pred`

In [159]:
fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': fishing_mort_pred.time.min()}] = (params['fish_mort_pred']*
                                               np.array(params['effort'][0]))

fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': fishing_mort_det.time.min()}] = (params['fish_mort_detritivore']*
                                              np.array(params['effort'][0]))

In [310]:
#output fisheries catches per yr at size - predators (Y_u)
catch_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 sel(time = fishing_mort_pred.time.min())*
 predators.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 sel(time = predators.time.min())*size_bin_vals[int(ind_min_fish_pred-1): -1])

#output fisheries catches per yr at size - detritivores (Y_v)
catch_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 sel(time = fishing_mort_pred.time.min())*
 detritivores.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 sel(time = predators.time.min())*size_bin_vals[int(ind_min_fish_det-1): -1])

In [237]:
if temp_effect:
    #Adding time dimension to temperature effect for pelagic group
    pel_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*(np.array(params['sea_surf_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input.time})    
    
    #Adding time dimension to temperature effect for benthic group
    ben_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*(np.array(params['sea_floor_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input.time})
else:
    pel_tempeffect = 1
    ben_tempeffect = 1

#To be applied to feeding rates for pelagics and benthic groups
feed_multiplier = params['hr_volume_search']*10**(log10_size_bins*metabolic_req_pred)

# Ingested food
growth_prop = 1-np.array(params['defecate_prop'])
# High quality food
high_prop = 1-np.array(params['def_low'])

/jobfs/128584439.gadi-pbs/ipykernel_2031461/2410198886.py:3: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  pel_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
/jobfs/128584439.gadi-pbs/ipykernel_2031461/2410198886.py:9: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the 

## Start of loop along time dimension

In [242]:
# for i, t in enumerate(dbpm_input.time):
#     print(i, t)
i = 0
t = dbpm_input.time[0]
ts = time[i]

In [243]:
ts

Timestamp('1840-12-01 00:00:00')

### Calculate growth and mortality

In [312]:
# feeding rates pelagics yr-1 (f_pel)
pred_growth = ((predators.sel(time = ts)*
                params['log_size_increase']).values@constant_growth)
feed_rate_pel = (pel_tempeffect.sel(time = t).values*\
                 (((feed_multiplier*params['pref_pelagic'])*pred_growth)/
                  (1+params['handling']*((feed_multiplier*params['pref_pelagic'])
                                         *pred_growth))))

# feeding rates benthics yr-1 (f_ben)
detrit_growth = ((detritivores.sel(time = ts)*
                  params['log_size_increase']).values@constant_growth)
feed_rate_bent = (pel_tempeffect.sel(time = t).values*\
                 (((feed_multiplier*params['pref_benthos'])*detrit_growth)/
                  (1+params['handling']*((feed_multiplier*params['pref_benthos'])
                                         *detrit_growth))))

# feeding rates detritivores yr-1 (f_det)
detritus_multiplier = ((1/size_bin_vals)*params['hr_vol_filter_benthos']*
                       10**(log10_size_bins*metabolic_req_pred)*
                       detritus.sel(time = ts).values)
feed_rate_det = (ben_tempeffect.sel(time = t).values*detritus_multiplier/
                 (1+params['handling']*detritus_multiplier))

# Predator growth integral yr-1 (GG_u)
growth_int_pred.loc[{'time': ts}] = (growth_prop*np.array(params['growth_pred'])*feed_rate_pel+
                                     high_prop*np.array(params['growth_detritivore'])*feed_rate_bent)

# Reproduction yr-1 (R_u)
if params['dynamic_reproduction']:
    reprod_pred.loc[{'time': ts}] = (growth_prop*(1-(np.array(params['growth_pred'])+params['energy_pred']))*
                                     feed_rate_pel+growth_prop*(1-(np.array(params['growth_detritivore'])+
                                                                   params['energy_detritivore']))*feed_rate_bent)

In [330]:
test = feed_rate_pel/((feed_multiplier*params['pref_pelagic'])*
               (predators.sel(time = ts)*params['log_size_increase']).values@constant_growth)

/jobfs/128584439.gadi-pbs/ipykernel_2031461/815649057.py:1: RuntimeWarning: invalid value encountered in divide
  test = feed_rate_pel/((feed_multiplier*params['pref_pelagic'])*


In [334]:
[t if frp > 0 for frp, t in zip(feed_rate_pel, test)]

0.0 nan
2.4629874593654004 0.34113052780717007
5.211056259282258 0.37647230710238355
8.259230603607572 0.4163586534821534
11.617012411873027 0.46121618187719493
15.287247227172541 0.5114739973179777
19.265123083236112 0.5675526573490183
23.53737453541436 0.6298506313437188
28.08176006029634 0.6987282871303512
32.86687148758998 0.7744896252540582
37.852319216473674 0.857362217398022
42.98931738069642 0.9474760779713802
48.22167006037023 1.0448424872354938
53.4871347181948 1.1493340610171214
58.71911418211846 1.2606675873486533
63.84860574182772 1.3783912798773312
68.80631722810104 1.501878087555529
73.5248469740064 1.6303265145841477
77.94081852541193 1.7627700264103014
81.99686248976771 1.8980955555653631
85.64334693360826 2.0350709150235136
88.83977354027004 2.17238014635544
91.55577800423305 2.308665067345857
93.77169809215222 2.442570639431235
95.47869939123055 2.5727913406301632
96.67847487163341 2.6981155694868777
97.3825580273719 2.8174652456892484
97.61130886617022 2.92992819512

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [319]:
feed_rate_pel/((feed_multiplier*params['pref_pelagic'])*
               (predators.sel(time = ts)*params['log_size_increase']).values@constant_growth)

/jobfs/128584439.gadi-pbs/ipykernel_2031461/1004970558.py:1: RuntimeWarning: invalid value encountered in divide
  feed_rate_pel/((feed_multiplier*params['pref_pelagic'])*


array([           nan, 3.41130528e-01, 3.76472307e-01, 4.16358653e-01,
       4.61216182e-01, 5.11473997e-01, 5.67552657e-01, 6.29850631e-01,
       6.98728287e-01, 7.74489625e-01, 8.57362217e-01, 9.47476078e-01,
       1.04484249e+00, 1.14933406e+00, 1.26066759e+00, 1.37839128e+00,
       1.50187809e+00, 1.63032651e+00, 1.76277003e+00, 1.89809556e+00,
       2.03507092e+00, 2.17238015e+00, 2.30866507e+00, 2.44257064e+00,
       2.57279134e+00, 2.69811557e+00, 2.81746525e+00, 2.92992820e+00,
       3.03478155e+00, 3.13150520e+00, 3.21978507e+00, 3.29950693e+00,
       3.37074186e+00, 3.43372512e+00, 3.48883037e+00, 3.53654119e+00,
       3.57742179e+00, 3.61208865e+00, 3.64118429e+00, 3.66535421e+00,
       3.68522766e+00, 3.70140238e+00, 3.71443342e+00, 3.72482572e+00,
       3.73303010e+00, 3.73944202e+00, 3.74440278e+00, 3.74820233e+00,
       3.75108334e+00, 3.75324601e+00, 3.75485323e+00, 3.75603573e+00,
       3.75689706e+00, 3.75751819e+00, 3.75796164e+00, 3.75827508e+00,
      

In [290]:
[frp/10 if frp > 0 else 0 for frp in feed_rate_pel]

[0,
 0.24629874593654005,
 0.5211056259282258,
 0.8259230603607571,
 1.1617012411873027,
 1.528724722717254,
 1.9265123083236113,
 2.353737453541436,
 2.808176006029634,
 3.286687148758998,
 3.7852319216473673,
 4.298931738069642,
 4.822167006037023,
 5.34871347181948,
 5.871911418211846,
 6.384860574182772,
 6.8806317228101035,
 7.352484697400639,
 7.794081852541193,
 8.199686248976771,
 8.564334693360825,
 8.883977354027005,
 9.155577800423305,
 9.377169809215221,
 9.547869939123055,
 9.66784748716334,
 9.73825580273719,
 9.761130886617021,
 9.739264618993563,
 9.67606077824541,
 9.575382220093335,
 9.441397227668439,
 9.278432202404542,
 9.090836660008133,
 8.88286505805613,
 8.658578447340364,
 8.421767432892441,
 8.17589655742582,
 7.924069057758248,
 7.669010042139966,
 7.4130655128181715,
 7.158214307417173,
 6.9060899280953,
 6.65800932797182,
 6.415005981138492,
 6.17786492481569,
 5.947157881835535,
 5.723277006928947,
 5.506466218013144,
 5.29684944987394,
 5.094455487430385

0.7